In [1]:
import os
import sys 
import pandas as pd 
import seaborn as sns

In [2]:
sys.path.append("../")
sys.path.append("../../")

In [3]:
from src.utils.TableConverter import md

class Falcon():

    def __init__(self, dir_path) -> None:
        self.dir_path = dir_path

In [4]:
path = "/scratch/work/koutchc1/datasets/falconcode/falconcode_v1_table_problems.csv"
df = pd.read_csv(path)
df

,id,course_id,type,exam,prompt,testcase,max_score,input_str,input_cast,output,...,loop_elements,loop_nested,stat_calculate,file_read,file_write,list,list_2d,dictionary,item_set,tuple
0,a3_3_animals,2,skill,1,<div>\n<p>&nbsp;</p>\n<p><strong>PROBLEM STATE...,"from cs110 import autograder\r\nimport random,...",100,0,0,1,...,1,0,0,0,0,1,0,0,0,0
1,a3_1_cargo,2,skill,1,<div>\n<p>&nbsp;</p>\n<p><strong>PROBLEM STATE...,"from cs110 import autograder\nimport random, m...",100,0,1,1,...,0,0,0,0,0,1,0,0,0,0
2,a3_3_games,2,skill,1,<div>\n<p>&nbsp;</p>\n<p><strong>PROBLEM STATE...,"from cs110 import autograder\r\nimport random,...",100,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,a3_4_heights,2,skill,1,<div>\n<p>&nbsp;</p>\n<p><strong>PROBLEM STATE...,"from cs110 import autograder\r\nimport random,...",100,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,pex3,2,project,0,"<div>\n <h2 style=""text-align: center;"">Com...","from cs110 import autograder\r\nimport random,...",0,0,0,1,...,1,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,a1_1_icbm,4,skill,1,<div>\n<p>&nbsp;</p>\n<p><strong>PROBLEM STATE...,"#import cs110,\r\nimport random, math\r\nfrom ...",100,1,1,0,...,0,0,0,0,0,0,0,0,0,0
657,a1_1_dorms,4,skill,1,<div>\n<p>&nbsp;</p>\n<p><strong>PROBLEM STATE...,"from cs110 import autograder\nimport random, m...",100,1,1,0,...,0,0,0,0,0,0,0,0,0,0
658,a1_1_cadet,4,skill,1,<div><strong>PROBLEM STATEMENT:<br></strong></...,"#import cs110,\r\nimport random, math\r\nfrom ...",100,1,1,0,...,0,0,0,0,0,0,0,0,0,0
659,a1_1_bases,4,skill,1,<div>\n<p>&nbsp;</p>\n<p><strong>PROBLEM STATE...,"from cs110 import autograder\r\nimport random,...",100,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def load_subset_with_missing_data(df):
    # Remove duplicate ids for project (same content, multiple semesters)
    df = df.drop_duplicates("id")
    df = df.fillna("")
    df = df.sort_values(by="id")
    # We do not have access to the external ressources (all the projects full description)
    # for assignments of type project, and they will be graded manually by instructors anyway 
    df = df[df.type != "project"]
    # So far, we do not care about the skills for annotation
    df = df.loc[:, :'max_score']
    # Format the prompt for readability
    df["prompt"] = df["prompt"].apply(md)
    df = df[["You have been provided with" in prompt for prompt in df["prompt"]]]
    df = df.reset_index(drop=True)
    return df

sub_df = load_subset_with_missing_data(df)

In [45]:
print(df[[".csv" in testcase for testcase in df["testcase"]]].prompt.iloc[0])

<html><head/><body><p><span style=" font-weight:600;">PROBLEM STATEMENT:<br/></span></p><p>You have been provided with crashdata_subset1.csv file. Here is a typical line from the file:</p><p align="center"><span style=" font-family:'Courier New';">2015, 1, 5, 0, 2-Car, No Injury, Following Too Closely, W 3rd</span></p><p>Each position in the line represents a different &quot;data element&quot;. Here is a key for your reference:</p><table border="0.5" style=" margin-top:0px; margin-bottom:0px; margin-left:36px; margin-right:36px;" cellspacing="3" cellpadding="0"><thead><tr><td/><td><p align="center"><span style=" font-weight:600;">Year</span></p></td><td><p align="center"><span style=" font-weight:600;">Month</span></p></td><td><p align="center"><span style=" font-weight:600;">Day</span></p></td><td><p align="center"><span style=" font-weight:600;">Hour</span></p></td><td><p align="center"><span style=" font-weight:600;">Accident Type</span></p></td><td><p align="center"><span style=" f

In [18]:
# Annotation loop
from tqdm.notebook import tqdm

def annotate(missing_data_df):
    missing = []
    for i, row in tqdm(missing_data_df.iterrows()):
        
        print("**TESTING SCRIPT**")
        print(row["testcase"])
        print()
        print(row["prompt"])
        print()
        
        missing_files = input("Missing files (including startercode)")
        missing.append({"files": missing_files})
        print("------------")
        print()
            
    missing = pd.DataFrame(missing)
    print(missing)
    
    return pd.concat([missing_data_df, missing], axis=1)

In [34]:
def extract_what_was_provided(statement, n_words=5):
    start = statement.find("You have been provided with")
    provided = statement[start:].split(" ")[5:]
    if provided[0] == 'a': provided = provided[1:] 
    provided = provided[:n_words]
    provided = " ".join(provided)
    return provided

In [40]:
provided.unique()

array(['function: get\\_value(n). Your',
       'dataset called\xa0`fuelprices.csv`\xa0that\n contains',
       'partially complete Python', 'template that gets',
       'function called calculate\\_equation,',
       'function called calculate\\_volume,',
       'function called calculate\\_future\\_value,',
       'function called calculate\\_hypotenuse,',
       'function called calculate\\_surface,', 'program that gets',
       'Python program that', 'dataset containing information',
       'list called\xa0`list_of_bases`. Write',
       'list called\xa0`list_of_cargo_planes`. Write',
       'list called\xa0`list_of_usaf_commands`. Write',
       'list called\xa0`list_of_coalition_countries`. Write',
       'list called\xa0`list_of_aircraft`. Write',
       'list called\xa0`list_of_medals` that',
       'list called `list_of_cars`.', 'list called `list_of_cities`.',
       'list called\xa0`department_list`. Write',
       'list called `grammy_list`.',
       'list called\xa0`list_o

In [113]:
def map_provided_to_category(subdf):
    subdf["extract"] = sub_df.prompt.apply(extract_what_was_provided)
    categories = ["function", "program", "file", "data_structure"]
    missing = []
    for provided, testcase in subdf[["extract", "testcase"]].to_numpy():
        file = ""
        words = testcase.split(" ")
        file = list(set([w for w in words if '.csv' in w or '.txt' in w]))
        if not file:
            words = provided.split(" ")
            file = list(set([w for w in words if '.csv' in w or '.txt' in w]))
        
        file = ",".join(file) if file else "startercode"
        information = {"name": file}
        missing.append(information)
        
    return pd.concat([subdf, pd.DataFrame(missing)], axis=1)

In [114]:
missing_data_df = map_provided_to_category(sub_df)
missing_data_df

,id,course_id,type,exam,prompt,testcase,max_score,provided,extract,name
0,PA2_practice2,4,lab,0,**PROBLEM STATEMENT:**You have been provided w...,"from cs110 import autograder\r\nimport random,...",100,function: get\_value(n). Your,function: get\_value(n). Your,startercode
1,PA2_practice4,4,lab,0,**PROBLEM STATEMENT:**\nYou have been provided...,"from cs110 import autograder\r\nimport random,...",100,dataset called `fuelprices.csv` that\n contains,dataset called `fuelprices.csv` that\n contains,called `fuelprices.csv` that\n
2,a1_3_arrow,3,skill,1,**PROBLEM STATEMENT:**\nYou have been provided...,"from cs110 import autograder\r\nimport random,...",100,partially complete Python,partially complete Python,startercode
3,a1_3_cards,3,skill,1,**PROBLEM STATEMENT:**\nYou have been provided...,"from cs110 import autograder\r\nimport random,...",100,partially complete Python,partially complete Python,startercode
4,a1_3_goldilocks,3,skill,1,**PROBLEM STATEMENT:**\nYou have been provided...,"from cs110 import autograder\r\nimport random,...",100,partially complete Python,partially complete Python,startercode
...,...,...,...,...,...,...,...,...,...,...
128,lsn9_callme,4,lab,0,**PROBLEM STATEMENT:**You have been provided w...,"from cs110 import autograder\r\nimport random,...",100,three functions. Open,three functions. Open,startercode
129,lsn9_practice1,2,lab,0,**PROBLEM STATEMENT:**You have been provided w...,"from cs110 import autograder\r\nimport random,...",100,program that prints,program that prints,startercode
130,lsn9_practice2,2,lab,0,**PROBLEM STATEMENT:**You have been provided w...,"from cs110 import autograder\nimport random, m...",100,program that gets,program that gets,startercode
131,lsn9_skill1,4,skill,0,**PROBLEM STATEMENT:**You have been provided w...,"from cs110 import autograder\r\nimport random,...",100,function called `draw_tree()`,function called `draw_tree()`,startercode


In [117]:
missing_data_df = missing_data_df[["id", "course_id", "type", "extract", "name"]]
missing_data_df

,id,course_id,type,extract,name
0,PA2_practice2,4,lab,function: get\_value(n). Your,startercode
1,PA2_practice4,4,lab,dataset called `fuelprices.csv` that\n contains,called `fuelprices.csv` that\n
2,a1_3_arrow,3,skill,partially complete Python,startercode
3,a1_3_cards,3,skill,partially complete Python,startercode
4,a1_3_goldilocks,3,skill,partially complete Python,startercode
...,...,...,...,...,...
128,lsn9_callme,4,lab,three functions. Open,startercode
129,lsn9_practice1,2,lab,program that prints,startercode
130,lsn9_practice2,2,lab,program that gets,startercode
131,lsn9_skill1,4,skill,function called `draw_tree()`,startercode


In [118]:
missing_data_df.to_csv("problems_df_missing_information.csv")

In [120]:
missing_data_df.name.unique()

array(['startercode', 'called\xa0`fuelprices.csv`\xa0that\n',
       'open("pokemon.csv",', 'open("powerplants.csv",',
       'open("legos.csv",', '"cadets.csv"\r\n',
       'called\xa0`afpft_v1.csv`\xa0that',
       'called\xa0`afpft_v2.csv`\xa0that', 'sample.txt.',
       'called\xa0`fuelprices.csv`\xa0that',
       'run_test("crashdata_2015.csv")\r\n,run_test("crashdata_2003.csv"),run_test("crashdata_2011.csv")',
       '["article.txt",,["article2.txt",,solution("article2.txt",,solution("article.txt",,solution("review.txt",,["review.txt",',
       'oscar\\_age\\_male.csv,'], dtype=object)

### Analysis of which tables are useful

In [ ]:
print(df.loc[df.problem_id == "Airstrike"]["prompt"])
print(df.loc[df.problem_id == "Airstrike"]["testcase"])

In [ ]:
path = "/scratch/work/koutchc1/datasets/falconcode/falconcode_v1_merged.csv"
runs_df = pd.read_csv(path, index_col=0)
runs_df

In [ ]:
print(runs_df.loc[df.problem_id == "Airstrike"]["source_code"].iloc[0])